# Using DeMinEr to identify base substitutions at low frequency

DeMinEr is a worflow to efficiently infer somatic mutation statistics within a sequenced region. Error-correction is performed with the use of mutation-free control samples (2) that provides an accurate and reproducible sequencing error baseline (position- and nucleotide-wise). Nucleotide counts (test and mutation-free control samples) are required as csv tabular files (sep=';') in the nucleotideCounts sub-directory. Working directory should contain the  Python source files (DeMinEr.py and DeMinErReport.py) and the reference sequence fasta file (gene.fasta).

## Run DeMinEr step-by-step

(Required packages : os, re, biopython, numpy, matplotlib)

### Load useful packages and functions, define test and control samples, gene and region

In [ ]:
from DeMinEr import *
%matplotlib inline

sample='WT-GC'
sample_Ctrl1='AID-1'
sample_Ctrl2='AID-2'
gene='Cd83m'
#region is range included in [0:length(gene)]; used to crop analyzed regions to uniformly sequenced regions (in depth)
region=range(100,1000,1) 

### Identify AID hotspots on target sequence

In [ ]:
(HotSpot_start_fw,HotSpot_start_rv) = getHotspots(gene)
HotSpotsregion=[i for i in set(HotSpot_start_fw+HotSpot_start_rv) if i in region]

### Compute alternative (non-reference) nucleotide frequencies and counts in sample (raw) and both controls

In [ ]:
Test,Testd=getAltFreq(sample,gene)
CTRL1,CTRL1d=getAltFreq(sample_Ctrl1,gene)
CTRL2,CTRL2d=getAltFreq(sample_Ctrl2,gene)
Mutcounts=Test*np.transpose([Testd,Testd,Testd,Testd])

print('Raw Mutation rate: '+str(round(np.sum(Mutcounts[region])/np.sum(np.array(Testd)[region])*1e5,1))+' per 100kb')
print('Mutation enrichment in AID-hotspots: '\
    +str(round(np.sum(Mutcounts[HotSpotsregion])/np.sum(Mutcounts[region])/(len(HotSpotsregion)/len(region)),2)))
    

### Compute processed mutation frequencies using DeMinEr Filter/error-correction steps

In [ ]:
TestF=filterTestvsControl(Test,Testd,CTRL1,CTRL2,sample,gene,region,t=3.29,foldChange=1.5)
MutcountsF=TestF*np.transpose([Testd,Testd,Testd,Testd])

print('Mutation rate: '+str(round(np.sum(MutcountsF[region])/np.sum(np.array(Testd)[region])*1e5,1))+' per 100kb')
print('Mutation enrichment in AID-hotspots: '\
    +str(round(np.sum(MutcountsF[HotSpotsregion])/np.sum(MutcountsF[region])/(len(HotSpotsregion)/len(region)),2)))

### Compute mutation profile (composition-corrected) from processed data

In [ ]:
mutationProfile=computeMutationProfile(MutcountsF,Testd,gene,region)
for i in range(5):
    print("\t".join([str(j) for j in mutationProfile[i]]))

In [ ]:
print('Raw mutation frequency in '+gene+' / '+sample)
plotMutations(Test,Testd,gene,region,'Test_'+sample+'_raw',display=True)
print('Processed mutation frequency in '+gene+' / '+sample)
plotMutations(TestF,Testd,gene,region,'Test_'+sample+'_filter',display=True)

## Generate an all-in-one DeMinEr report

(Required packages : wand, reportlab)

In [ ]:
from DeMinErReport import *
DeMinEr2Report('WT-GC','AID-1','AID-2','Cd83m',region=range(100,1000,1))

In [ ]:
from DeMinErReport import *
DeMinEr2Report('UNG-GC','AID-1','AID-2','Cd83m',region=range(100,1000,1))